## SleepPoseNet
The code below is the example of running *Experiment IV* in the paper that using *Dataset II*. Other different settings in the paper can be done by modifying this code. If you have any enquiry, 


In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from IPython.display import HTML
from scipy.fftpack import fft, fftshift
from scipy.signal import butter, lfilter
import matplotlib.lines as mlines
import math
from numpy import dstack
import numpy.matlib
from pandas import read_csv
from skimage import img_as_ubyte
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import precision_recall_fscore_support
from skimage import filters
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.externals import joblib
from sklearn.model_selection import PredefinedSplit

from tensorflow.keras import layers
from tensorflow.keras import models
from tensorflow.keras.layers import Dense, Activation, BatchNormalization, UpSampling2D, Flatten, Input, Conv2D, Conv2DTranspose, LeakyReLU, PReLU, Lambda, add
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from tensorflow.keras.utils import to_categorical
import scipy
from scipy.fftpack import fft
from scipy.fftpack import fft2
from scipy import signal
from scipy.interpolate import CubicSpline
from collections import Counter
from sklearn.metrics import accuracy_score

#Essential parameters
fBin = 0 #First range bin
lBin = 180 #Last range bin
nBin = 40 #The total number of range bins
time_window = 100 #The total number of slow time indices

#WRTFT PARAMETERS
NFFT = 25
stride  = 2
outdim_size = 20

#Hyperparameter of CNN
PATIENCE = 200 #Early stopping
EPOCHS = 1000 
BATCH_SIZE = 16


# CNN Model

In [ ]:

#SleepPoseNet Model
def make_model():

  #TD input
  amp_input = layers.Input(shape=(nBin, 99,1), name = 'amp_input')
  x_amp = layers.Conv2D(16, (2, 3), activation='relu', padding='valid', strides = (1,1), kernel_regularizer=tf.keras.regularizers.l2(0.0005))(amp_input)
  x_amp = layers.SpatialDropout2D(0.3)(x_amp)
  x_amp = layers.MaxPooling2D((2, 3))(x_amp)
  x_amp = layers.Conv2D(32, (2, 3),  activation='relu', padding='valid', strides = (1,1), kernel_regularizer=tf.keras.regularizers.l2(0.0005))(x_amp)
  x_amp = layers.SpatialDropout2D(0.3)(x_amp)
  x_amp = layers.MaxPooling2D((2, 3))(x_amp)
  x_amp = layers.Conv2D(32, (2, 3),  activation='relu', padding='valid', strides = (1,1), kernel_regularizer=tf.keras.regularizers.l2(0.0005))(x_amp)
  x_amp = layers.SpatialDropout2D(0.3)(x_amp)
  x_amp = layers.MaxPooling2D((2, 3))(x_amp)
  x_amp = layers.Flatten()(x_amp)
  amp_flat_shape = x_amp.shape
  x_amp = layers.Dense(outdim_size, kernel_regularizer=tf.keras.regularizers.l2(0.01))(x_amp)

  #WRTFT Input
  wrtft_input = layers.Input(shape=(NFFT//2+1, (time_window-NFFT)//stride+1,1))
  x_wrtft = layers.Conv2D(10, (2,2), activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(0.0005))(wrtft_input)
  x_wrtft = layers.SpatialDropout2D(0.3)(x_wrtft)
  x_wrtft = layers.MaxPooling2D((2, 2))(x_wrtft)
  x_wrtft = layers.Conv2D(20, (2, 2), activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(0.0005))(x_wrtft)
  x_wrtft = layers.SpatialDropout2D(0.3)(x_wrtft)
  x_wrtft = layers.MaxPooling2D((2, 2))(x_wrtft)
  x_wrtft = layers.Flatten()(x_wrtft)
  wrtft_flat_shape = x_wrtft.shape
  x_wrtft = layers.Dense(outdim_size, kernel_regularizer=tf.keras.regularizers.l2(0.01))(x_wrtft)


  #Fuse two features
  concat_output = layers.concatenate([x_amp, x_wrtft], name='cca_output')

  #ouput postures = 5 classes
  output_posture = layers.Dense(10, kernel_regularizer=tf.keras.regularizers.l2(0.1))(concat_output)
  output_posture = layers.ReLU()(output_posture)
  
  output_posture = layers.Dropout(0.5)(output_posture)
  output_posture = layers.Dense(5, activation='softmax', name='posture_output')(output_posture)

  dcnn_model = Model(inputs=[amp_input, wrtft_input], outputs=output_posture)

  return dcnn_model


## Data Preprocessing

In [ ]:
#Preprocessing functions
#Remove DC noise
def time_avg(x):
  x_temp = np.copy(x)
  for i in range(x.shape[0]):
    for s in range(x.shape[1]):
      x_avg = x_temp[i,s,:].mean()
      x[i, s, :] = x[i,s,:] - x_avg
  return x

#Remove scatter effect from other objects
def range_avg(x):
  x_temp = np.copy(x)
  for i in range(x.shape[0]):
    for s in range(x.shape[2]):
      x_avg = x_temp[i,:,s].mean()
      x[i, :, s] = x[i,:,s] - x_avg
  return x

#Range Selection
def crop_by_highest_sum(X, X_diff, spatial_size):
  spatial_highest_position = list()
  spatial_end = X_diff.shape[1]-spatial_size+1
  spatial_images_crop = list()

  #Spatial crop
  for i in range (X.shape[0]):
    spatial_max_value = -1000
    for j in range (10, spatial_end):
      current_sum = np.sum(X_diff[i, j:j+spatial_size, :]**2)
      if current_sum>=spatial_max_value:
        spatial_position = j
        spatial_max_value = current_sum
    spatial_images_crop.append(X[i, spatial_position:spatial_position+spatial_size, :])
    spatial_highest_position.append(spatial_position)

  X_crop = np.array(spatial_images_crop)
  return X_crop, spatial_highest_position

## Data Augmentation

Our code is modified from the [Original Code](https://github.com/terryum/Data-Augmentation-For-Wearable-Sensor-Data) which is used for wearable sensors



In [ ]:
#Data Augmentation functions
def signal_shift(x,shft_arr):
    res = []
    for s in shft_arr:
        zero = np.zeros((x.shape[0],abs(s)))
        tmp = x
        if s > 0:
            tmp = np.concatenate((zero,x[:,:-s]),axis = 1)
        elif s < 0:
            tmp = np.concatenate((x[:,-s:],zero),axis = 1)
        res.append(tmp)
    return res

def signal_shift_time(x,shft_arr):
    res = []
    x = x.T
    for s in shft_arr:
        zero = np.zeros((x.shape[0],abs(s)))
        tmp = x
        if s > 0:
            tmp = np.concatenate((zero,x[:,:-s]),axis = 1)
        elif s < 0:
            tmp = np.concatenate((x[:,-s:],zero),axis = 1)
        tmp = tmp.T
        res.append(tmp)
    return res

def GenerateRandomCurves(X, sigma, knot=4):
    xx = (np.ones((X.shape[1],1))*(np.arange(0,X.shape[0], (X.shape[0]-1)/(knot+1)))).transpose()
    yy = np.random.normal(loc=1.0, scale=sigma, size=(knot+2, 1))
    x_range = np.arange(X.shape[0])
    cs_x = CubicSpline(xx[:,0], yy[:,0])
    return np.array(cs_x(x_range)).transpose()

def DistortTimesteps(X, sigma):
    tt = GenerateRandomCurves(X, sigma) # Regard these samples aroun 1 as time intervals
    tt_cum = np.cumsum(tt, axis=0)        # Add intervals to make a cumulative graph
    # Make the last value to have X.shape[0]
    t_scale = (X.shape[0]-1)/tt_cum[-1]
    tt_cum = tt_cum*t_scale
    return tt_cum

def DA_TimeWarp(X, sigma):
    tt_new = DistortTimesteps(X, sigma)
    X_new = np.zeros(X.shape)
    x_range = np.arange(X.shape[0])
    for i in range(X.shape[1]):
        X_new[:,i] = np.interp(x_range, tt_new, X[:,i])
    return X_new

def DA_MagWarp(X, sigma):
  X_new = np.zeros(X.shape)
  for i in range (X.shape[1]):
    X_new[:, i] = X[:, i] * GenerateRandomCurves(X, sigma)
  return X_new
    

def increase_by_timewarp(dat_x, dat_y):  
  dat_x_aug = []
  dat_y_aug = []
  timewarp_array = [0, 0.4]

  for smp in dat_x:
    for rep in timewarp_array:
      if rep  != 0:
        dat_x_aug += [DA_TimeWarp(smp.T,rep)]
      else:
        dat_x_aug += [smp.T]

  for y in dat_y:
    dat_y_aug += [y]*len(timewarp_array)

  return np.swapaxes(np.array(dat_x_aug), 1 ,2), np.array(dat_y_aug)

def increase_by_magwarp(dat_x, dat_y):  
  dat_x_aug = []
  dat_y_aug = []
  magwarp_array = [0, 0.4]

  for smp in dat_x:
    for rep in magwarp_array:
      if rep  != 0:
        dat_x_aug += [DA_MagWarp(smp.T, rep)]
      else:
        dat_x_aug += [smp.T]

  for y in dat_y:
    dat_y_aug += [y]*len(magwarp_array)

  return np.swapaxes(np.array(dat_x_aug), 1 ,2), np.array(dat_y_aug)
  
def increase_by_shift_time(dat_x, dat_y):
  dat_x_aug = []
  dat_y_aug = []
  shft_array_time = [0,-5, 5, -10, 10]
  for smp in dat_x:
    dat_x_aug += signal_shift_time(smp.T,shft_array_time)

  for y in dat_y:
    dat_y_aug += [y]*len(shft_array_time)
  
  return np.swapaxes(np.array(dat_x_aug), 1 ,2), np.array(dat_y_aug)

def increase_by_shift_range(dat_x, dat_y):
  dat_x_aug = []
  dat_y_aug = []
  shft_array = [0,2,4]

  for smp in dat_x:
    dat_x_aug += signal_shift(smp.T,shft_array)

  for y in dat_y:
    dat_y_aug += [y]*len(shft_array)
  return np.swapaxes(np.array(dat_x_aug), 1 ,2), np.array(dat_y_aug)

## Weighted Range-Time-Frequency Transform (WRTFT)

In [ ]:

#WRTFT functions
def plot_wrtft(F):
    plt.figure(figsize=(10,4))
    plt.imshow(F, cmap='hot', aspect='auto', interpolation='catrom')
    plt.colorbar()
    plt.gca().invert_yaxis()
    plt.ylabel('Frequency')
    plt.xlabel('Time')
    plt.show()

def wrtft(X):
    R = X # select frame range 
    val = filters.threshold_otsu(R)
    B = np.copy(R)

    for i in range(R.shape[0]):
        for j in range(R.shape[1]):
            if R[i][j] < val:
                B[i][j] = 0
            else :
                B[i][j] = 1

    min_r = 200
    max_r = -1
    for i in range(B.shape[0]):
        for j in range(B.shape[1]):
            if B[i][j]:
                min_r = min(min_r,i)
                max_r = max(max_r,i)
    Em = []
    for i in range(R.shape[0]):
        Em.append(np.sum(R[i,:]**2))
    Em = np.array(Em)            
    sigma = Em[min_r:max_r]/sum(Em[min_r:max_r]) 

    F = np.zeros((NFFT//2+1,(R.shape[1]-NFFT)//stride+1))
    for m in range(min_r,max_r):
        _,_,arr = signal.stft(R[m,:], fs = 1, nperseg=NFFT, noverlap = NFFT - stride, padded = False, boundary = None)
        arr = np.abs(arr)
        F += sigma[m-min_r]*arr
        del arr
    return F
from tqdm import tqdm
def apply_wrtft(X):
    res = []
    cnt = 0
    for dat in tqdm(X,position = 0):
        F = wrtft(dat)
        res.append(F)
        cnt += 1
    return np.array(res)

## Data loading and preparation

In [ ]:


# Remove side-to-prone and prone-to-side classes (In the paper we use only 5 classes)
def remove_class(y):
  idx = list()
  for i in range(y.shape[0]):
    if not(y[i]==1 or y[i]==2):
      idx.append(i)
  return idx
def transform_class(y):
  # 0 = Supine side
  # 1 = Supine prone
  # 2 = Side supine
  # 3 = Prone supine
  # 4 = Background
  y_out = list()
  for i in range (len(y)):
    if y[i]==0: #Supine side
      y_out.append(0)
    elif y[i]==3: #Side supine
      y_out.append(2)
    elif y[i]==4: #Supine prone
      y_out.append(1)
    elif y[i]==5: #Prone supine
      y_out.append(3)
    elif y[i]==6: #Background
      y_out.append(4)

  return np.array(y_out)

#Load data
prefix='/content/drive/My Drive/Colab Notebooks/Sleep Radar/'
data_path=prefix+'2020 Data/radar_data/'
model_path =prefix+'model/best_model.h5'
dat_x_session1 = np.load(data_path+'X_wall_session1.npy')
dat_x_session2 = np.load(data_path+'X_wall_session2.npy')
y_session1 = np.load(data_path+'y_digit_session1.npy')
y_session2 = np.load(data_path+'y_digit_session2.npy')
sub_map_session1 = np.load(data_path+'subjects_session1.npy')
sub_map_session2 = np.load(data_path+'subjects_session2.npy')
dat_x = np.concatenate((dat_x_session1, dat_x_session2), axis=0)
y = np.concatenate((y_session1, y_session2), axis=0)
sub_map = np.concatenate((sub_map_session1, sub_map_session2)) 
session_label = np.array([1]*(y_session1.shape[0]) + [2]*(y_session2.shape[0]))
rmv_idx = remove_class(y) #Remove side prone and prone side
dat_x = dat_x[rmv_idx]
y = transform_class(y[rmv_idx])
sub_map = sub_map[rmv_idx]
session_label = session_label[rmv_idx]

# # preprocess X
X = np.swapaxes(dat_x,1,2)
X = X[:,:,30:130]
X_amp = np.abs(X)
X_amp = time_avg(X_amp)
X_amp = range_avg(X_amp)
X_amp_map = np.diff(X_amp, n=1, axis=2)
X_amp, positions = crop_by_highest_sum(X_amp, X_amp_map, nBin)
space, time = X_amp.shape[1], X_amp.shape[2]


###############Prepare data for 10 folds####################
#Prepare 10 fold datasets
# get all data for multiple subject
def data_for_subjects(sub_map, sub_id):
    # get row indexes for the subject id
    ix = [i for i in range(len(sub_map)) if sub_map[i] in sub_id]
    # return the selected samples
    return ix

sub_map_subjects = np.unique(sub_map)
sub_map_subjects_val = [[2,3,4,5],[3,4,5,6],[4,5,6,7],[5,6,7,8], [6,7,8,9],[7,8,9,10],[8,9,10,11],
                        [9,10,11,12],[10,11,12,1],[11,12,1,2], [12,1,2,3], [1,2,3,4]]
sub_map_subjects_test = [[1], [2], [3], [4], [5], [6], [7], [8], [9], [10], [11], [12]]
sub_map_subjects_train = []
fold = len(sub_map_subjects_test)

#Prepare data for 10-fold
for i in range (fold):
  train_temp =[]
  for j in range (1,13):
    if j not in sub_map_subjects_val[i] and j not in sub_map_subjects_test[i]:
      train_temp.append(j)
  sub_map_subjects_train.append(train_temp)


## The final section is for 10-fold experiments

In [ ]:
#10-fold loop  
for idx in range (0,fold):  
  train_idx = data_for_subjects(sub_map, sub_map_subjects_train[idx])
  val_idx = data_for_subjects(sub_map, sub_map_subjects_val[idx])
  test_idx = data_for_subjects(sub_map, sub_map_subjects_test[idx])
  session_label_test = session_label[test_idx]
  # # Choose train val test set
  x_amp_train = X_amp[train_idx]
  x_amp_val = X_amp[val_idx]
  x_amp_test = X_amp[test_idx]
  y_train = y[train_idx]
  y_val = y[val_idx]
  y_test =  y[test_idx]

  #Apply Data Augmentation
  x_amp_train, y_train = increase_by_shift_time(x_amp_train, y_train)
  x_amp_train, y_train = increase_by_shift_range(x_amp_train, y_train)
  x_amp_train, y_train = increase_by_timewarp(x_amp_train, y_train)
  x_amp_train, y_train = increase_by_magwarp(x_amp_train, y_train)

  #Applying TD
  x_diff_train = np.diff(x_amp_train, n=1,axis=2)
  x_diff_val = np.diff(x_amp_val, n=1, axis=2)
  x_diff_test = np.diff(x_amp_test, n=1, axis=2) 
  x_diff_train = x_diff_train.reshape(x_diff_train.shape[0], space*(time-1))
  x_diff_val = x_diff_val.reshape(x_diff_val.shape[0], space*(time-1))
  x_diff_test = x_diff_test.reshape(x_diff_test.shape[0], space*(time-1))


  #Appying WRTFT
  x_wrtft_train = apply_wrtft(x_amp_train)
  x_wrtft_val = apply_wrtft(x_amp_val)
  x_wrtft_test = apply_wrtft(x_amp_test)      
  range_bins, time_bins = x_wrtft_train.shape[1], x_wrtft_train.shape[2]
  x_wrtft_train = x_wrtft_train.reshape(-1, range_bins*time_bins)
  x_wrtft_val = x_wrtft_val.reshape(-1, range_bins*time_bins)
  x_wrtft_test = x_wrtft_test.reshape(-1, range_bins*time_bins)
  scaler_wrtft = StandardScaler()
  scaler_wrtft.fit(x_wrtft_train)
  x_wrtft_train = scaler_wrtft.transform(x_wrtft_train)
  x_wrtft_val = scaler_wrtft.transform(x_wrtft_val)
  x_wrtft_test = scaler_wrtft.transform(x_wrtft_test)
  x_wrtft_train = x_wrtft_train.reshape(-1, range_bins, time_bins, 1)
  x_wrtft_val = x_wrtft_val.reshape(-1, range_bins, time_bins, 1)
  x_wrtft_test = x_wrtft_test.reshape(-1, range_bins, time_bins, 1)


  scaler_diff = StandardScaler()
  scaler_diff.fit(x_diff_train)
  def preprocess(X):
    X = scaler_diff.transform(X)
    X = X.reshape(X.shape[0], space, time-1, 1).astype('float32')
    return X

  x_diff_train = preprocess(x_diff_train)
  x_diff_val = preprocess(x_diff_val)
  x_diff_test = preprocess(x_diff_test)

  
  #Loss function and optimizer for DCNN models
  loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
  optimizer = Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=1e-08)        

  #Multiview & Task DCNN
  modelNameMultiview = prefix + model_path
  @tf.function
  def Multiview_DCNN_train_step(x, y, model):
      with tf.GradientTape() as tape:
          predictions= model(x, training=True)
          posture_loss = loss_object(y, predictions)
          
          loss = posture_loss
          gradients = tape.gradient(loss, model.trainable_variables)
      
      optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  def Multiview_DCNN_train(epochs, batch_size):
      batch_count = int(x_diff_train.shape[0] / batch_size)
      model = make_model()

      best_epoch = 0
      best_loss_val = 100000
      count = 0
      for e in range(1, epochs+1):
          # print ('-'*15, 'Epoch %d' % e, '-'*15)
          if count>PATIENCE:
              break
          for i in range(batch_count):

              rand_nums = np.random.randint(0, x_diff_train.shape[0], size=batch_size)
              x_diff_batch = tf.convert_to_tensor(x_diff_train[rand_nums])
              x_wrtft_batch = tf.convert_to_tensor(x_wrtft_train[rand_nums])
              y_batch = tf.convert_to_tensor(y_train[rand_nums])
              

              Multiview_DCNN_train_step([x_diff_batch, x_wrtft_batch], y_batch, model)
              

          #Compute loss for validation set
          predictions_val = model([x_diff_val, x_wrtft_val], training=False)
          loss_val = loss_object( tf.convert_to_tensor(y_val), predictions_val)
          if loss_val<best_loss_val:
              count=0
              best_epoch = e
              best_loss_val = loss_val
              # print(best_loss_val)
              model.save(modelNameMultiview)
              # print('Model saved')
          else:
              count+=1

  Multiview_DCNN_train(EPOCHS, BATCH_SIZE)
  saved_multiview = load_model(modelNameMultiview, compile=False)
  predictions_train = saved_multiview.predict([x_diff_train, x_wrtft_train], batch_size=128)
  predictions_train = np.argmax(predictions_train, axis=1)

  predictions_val = saved_multiview.predict([x_diff_val, x_wrtft_val])
  predictions_val = np.argmax(predictions_val, axis=1)

  predictions_test= saved_multiview.predict([x_diff_test, x_wrtft_test])
  predictions_test = np.argmax(predictions_test, axis=1)

  print('Train Acc=', accuracy_score(y_train, predictions_train))
  print('Val Acc=', accuracy_score(y_val, predictions_val))
  print('Session1 Test Acc=', accuracy_score(y_test[0:34], predictions_test[0:34]))
  print('Session2 Test Acc=', accuracy_score(y_test[34:], predictions_test[34:]))
  cm = confusion_matrix(y_test, predictions_test)
  print(cm)